[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SociallyIneptWeeb/AICoverGen/blob/main/AICoverGen_colab.ipynb)

In [ ]:
#@title Clone repository
!git clone https://github.com/SociallyIneptWeeb/AICoverGen.git
%cd AICoverGen

In [ ]:
#@title Install requirements
!pip install -q -r requirements.txt

In [ ]:
#@title Download MDXNet Vocal Separation and Hubert Base Models
!python src/download_models.py

In [ ]:
import os
#@title Download RVC model

#@markdown ## Download a RVC model from a HuggingFace link
#@markdown Sample Selena Gomez model by jooheons on discord
MODEL_LINK = 'https://huggingface.co/jooheons/selenagv2/resolve/main/selenagv2.zip' #@param {type:"string"}
#@markdown ## Give a suitable name for the voice model to use later
model_name = 'Selena' #@param {type:"string"}
zip_name = MODEL_LINK.split('/')[-1]
!wget {MODEL_LINK}
!unzip {zip_name} -d rvc_models/{model_name}
!rm {zip_name}

In [ ]:
import re
import ffmpeg
from IPython.display import Audio, display

#@title Run Cover Generation (only preview first 30s of cover)
#@markdown Enter the voice model to use, same as entered in the above cell.
model_name = 'Selena' #@param {type:"string"}

#@markdown Enter a link to a song on youtube
yt_link = 'https://www.youtube.com/watch?v=FSXuM2v0YLY' #@param {type:"string"}
match = re.search(r"v=([^&]+)", yt_link)
song_id = match.group(1)

#@markdown Enter the pitch change to add to the vocals. Generally use 12 for Male to Female voice conversions, and -12 for Female to Male conversions
pitch_change = 0 #@param {type:"number"}

!python src/main.py -yt {yt_link} -dir {model_name} -p {pitch_change}

cover_dir = os.path.join(os.getcwd(), 'song_output', song_id)
cover_path = ''
for filepath in os.listdir(cover_dir):
  if filepath.endswith(f'({model_name} Ver).wav'):
    cover_path = os.path.join(cover_dir, filepath)
    break

audio_input = ffmpeg.input(cover_path)
audio_cut = audio_input.audio.filter('atrim', duration=30)
audio_output = ffmpeg.output(audio_cut, 'test.wav')
ffmpeg.run(audio_output, overwrite_output=True)
Audio('preview.wav')

In [ ]:
#@title Download the full AI Generated Cover
from google.colab import files
files.download(cover_path)